In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import os
from keras.layers import *
from keras.models import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping


In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 2. Path to the dataset
TRAIN_PATH = '/content/drive/MyDrive/brain tumor dataset/train'
TEST_PATH = '/content/drive/MyDrive/brain tumor dataset/test'


In [ ]:
# 3. Create the Image Data Generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(128, 128),  # Reduced image size for efficiency
    batch_size=16,  # Smaller batch size
    class_mode='categorical'
)

Found 5711 images belonging to 4 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(128, 128),  # Reduced image size
    batch_size=16,  # Smaller batch size
    class_mode='categorical'
)

Found 1311 images belonging to 4 classes.


In [ ]:
# 4. Build the DenseNet201 model for classification
def DenseNet_classification_model(input_size=(128, 128, 3), num_classes=4):
    # Load the pre-trained DenseNet201 model from TensorFlow
    base_model = tf.keras.applications.DenseNet201(
        input_shape=input_size,
        include_top=False,
        weights='imagenet'
    )

    # Freeze the layers in the base model
    base_model.trainable = False

    # Add custom layers on top
    inputs = Input(input_size)
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Define the model
    model = Model(inputs, outputs)
    return model

In [ ]:
# Instantiate the DenseNet201 model for classification
model = DenseNet_classification_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# 5. Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [ ]:
# 6. Set the steps_per_epoch and validation_steps
steps_per_epoch = len(train_generator)
validation_steps = len(test_generator)


In [ ]:
# 7. Train the model
hist = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=15,
    validation_data=test_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


357/357 ━━━━━━━━━━━━━━━━━━━━ 1639s 4s/step - accuracy: 0.7554 - loss: 0.7040 - val_accuracy: 0.7773 - val_loss: 0.6597
Epoch 2/15


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/15


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


357/357 ━━━━━━━━━━━━━━━━━━━━ 635s 2s/step - accuracy: 0.8797 - loss: 0.3218 - val_accuracy: 0.8360 - val_loss: 0.4254
Epoch 4/15
357/357 ━━━━━━━━━━━━━━━━━━━━ 0s 302us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/15
357/357 ━━━━━━━━━━━━━━━━━━━━ 615s 2s/step - accuracy: 0.9097 - loss: 0.2485 - val_accuracy: 0.8490 - val_loss: 0.3701
Epoch 6/15
357/357 ━━━━━━━━━━━━━━━━━━━━ 0s 312us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/15
357/357 ━━━━━━━━━━━━━━━━━━━━ 602s 2s/step - accuracy: 0.9119 - loss: 0.2278 - val_accuracy: 0.8741 - val_loss: 0.3472
Epoch 8/15
357/357 ━━━━━━━━━━━━━━━━━━━━ 0s 311us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/15
357/357 ━━━━━━━━━━━━━━━━━━━━ 594s 2s/step - accuracy: 0.9257 - loss: 0.2008 - val_accuracy: 0.8917 - val_loss: 0.2520
Epoch 10/15
357/357 ━━━━━━━━━━━━━━━━━━━━ 0s 295us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/15
357/357 ━━━━━━━━━━━━━━━━━━━━ 622s 2s/step - accuracy: 0.9312 - loss: 0.1799 - val_accuracy: 0.9138

In [ ]:
# 8. Evaluate the model on the test data
evaluation = model.evaluate(test_generator)
print(f'Test Loss: {evaluation[0]}')
print(f'Test Accuracy: {evaluation[1]}')

82/82 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - accuracy: 0.9236 - loss: 0.1944
Test Loss: 0.21001659333705902
Test Accuracy: 0.9176201224327087
